In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import torch
import librosa
import librosa.display
import os
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
emotion = []
gender = []
actor = []
file_path = []
audio = "D:\\ML Projects\\Emotion-Aware Music Composer\\audio_speech_actors_01-24"
actor_folders = os.listdir(audio)

In [3]:
for i in actor_folders:
    file_name = os.listdir(audio + '\\' + i)
    for f in file_name:
        part = f.split('.')[0].split('-')
        emotion.append(part[2])
        actor.append(part[6])
        bg = int(part[6])
        if bg%2 == 0:
            bg = 'female'
        else:
            bg= 'male'
        gender.append(bg)
        file_path.append(audio + '\\' + i + '\\' + f)

In [17]:
audio_df = pd.DataFrame(emotion)
audio_df = audio_df.astype(int).replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_df = pd.concat([pd.DataFrame(gender),audio_df,pd.DataFrame(actor)],axis=1)
audio_df.columns = ['gender','emotion','actor']
audio_df = pd.concat([audio_df,pd.DataFrame(file_path, columns = ['path'])],axis=1)

In [18]:
audio_df.head()

,gender,emotion,actor,path
0,male,neutral,01,D:\ML Projects\Emotion-Aware Music Composer\au...
1,male,neutral,01,D:\ML Projects\Emotion-Aware Music Composer\au...
2,male,neutral,01,D:\ML Projects\Emotion-Aware Music Composer\au...
3,male,neutral,01,D:\ML Projects\Emotion-Aware Music Composer\au...
4,male,calm,01,D:\ML Projects\Emotion-Aware Music Composer\au...


In [19]:
df = pd.DataFrame(columns=['mel spectrogram'])
counter = 0

In [20]:
# ITERATE OVER ALL AUDIO FILES AND EXTRACT LOG MEL SPECTROGRAM MEAN VALUES INTO DF FOR MODELING 
df = pd.DataFrame(columns=['mel_spectrogram'])
counter=0
for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    #get the mel-scaled spectrogram (ransform both the y-axis (frequency) to log scale, and the “color” axis (amplitude) to Decibels, which is kinda the log scale of amplitudes.)
    spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128,fmax=8000) 
    db_spec = librosa.power_to_db(spectrogram)
    #temporally average spectrogram
    log_spectrogram = np.mean(db_spec, axis = 0)
        
    
    df.loc[counter] = [log_spectrogram]
    counter=counter+1
print(len(df))
df.head()

1440


,mel_spectrogram
0,"[-76.38517, -76.38517, -76.38517, -76.38517, -..."
1,"[-75.48293, -75.4404, -75.554375, -75.20429, -..."
2,"[-75.15079, -75.15079, -75.15079, -75.15079, -..."
3,"[-75.268524, -75.268524, -75.268524, -75.26852..."
4,"[-80.14777, -80.14777, -80.14777, -80.14777, -..."


In [21]:
df = df['mel_spectrogram'].apply(pd.Series)

C:\Users\HP\AppData\Local\Temp\ipykernel_10624\3619587865.py:1: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = df['mel_spectrogram'].apply(pd.Series)


In [22]:
df = pd.concat([audio_df, df], axis=1)
df.drop(columns=['path'], inplace=True)
df.head()

,gender,emotion,actor,0,1,2,3,4,5,6,...,249,250,251,252,253,254,255,256,257,258
0,male,neutral,01,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,male,neutral,01,-75.482933,-75.440399,-75.554375,-75.204292,-75.230873,-75.319717,-75.654137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,male,neutral,01,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male,neutral,01,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,male,calm,01,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,...,-80.147774,-80.147774,-80.122353,-79.998413,-80.119652,-80.147774,-80.130814,-80.015106,-80.147774,-80.147774


In [23]:
df

,gender,emotion,actor,0,1,2,3,4,5,6,...,249,250,251,252,253,254,255,256,257,258
0,male,neutral,01,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,male,neutral,01,-75.482933,-75.440399,-75.554375,-75.204292,-75.230873,-75.319717,-75.654137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,male,neutral,01,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male,neutral,01,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,male,calm,01,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,...,-80.147774,-80.147774,-80.122353,-79.998413,-80.119652,-80.147774,-80.130814,-80.015106,-80.147774,-80.147774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,female,surprise,24,-66.861359,-66.861359,-64.904015,-64.663872,-66.493340,-66.080956,-66.156181,...,-66.861359,-66.859650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1436,female,surprise,24,-62.385281,-62.385281,-62.385281,-62.385281,-62.385281,-62.385281,-62.385281,...,-62.381546,-62.385281,-62.385281,-62.347294,-62.385281,-62.385281,-62.347816,-62.358097,-62.385281,-62.383404
1437,female,surprise,24,-65.515617,-65.515617,-65.515617,-65.515617,-65.515617,-65.515366,-65.443626,...,-65.430115,-65.515617,-65.515617,-65.515617,-65.515617,-65.515617,-65.503281,-65.504234,-65.491219,-65.437935
1438,female,surprise,24,-63.501495,-63.501495,-63.421318,-62.750801,-59.717041,-55.641850,-52.376606,...,-62.756645,-62.907124,-62.763821,-62.752789,-63.128666,-63.439880,-63.484493,-63.420803,-63.217548,-63.179066


In [24]:
df.isnull().sum()

gender       0
emotion      0
actor        0
0            0
1            0
          ... 
254        316
255        316
256        377
257        377
258        377
Length: 262, dtype: int64

In [25]:
df.fillna(0, inplace=True)

In [27]:
df.head()

,gender,emotion,actor,0,1,2,3,4,5,6,...,249,250,251,252,253,254,255,256,257,258
0,male,neutral,01,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,-76.385170,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,male,neutral,01,-75.482933,-75.440399,-75.554375,-75.204292,-75.230873,-75.319717,-75.654137,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,male,neutral,01,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,-75.150787,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,male,neutral,01,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,-75.268524,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,male,calm,01,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,-80.147774,...,-80.147774,-80.147774,-80.122353,-79.998413,-80.119652,-80.147774,-80.130814,-80.015106,-80.147774,-80.147774


In [32]:
train, test= train_test_split(df, test_size=0.2, random_state=0,
                              stratify=df[['emotion','gender','actor']])

X_train = train.iloc[:, 3:]
y_train = train.iloc[:,:2].drop(columns=['gender'])
X_test = test.iloc[:,3:]
y_test = test.iloc[:,:2].drop(columns=['gender'])

In [33]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [34]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [36]:
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
y_test = to_categorical(lb.fit_transform(y_test))

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [43]:
# Define the model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)),  # First LSTM layer
    Dropout(0.3),  # Regularization
    
    LSTM(64, return_sequences=False),  # Second LSTM layer
    Dropout(0.3),
    
    Dense(32, activation='relu'),  # Fully connected layer
    Dense(y_train.shape[1], activation='softmax')  # Output layer (softmax for multi-class classification)
])

# Compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(learning_rate=0.0005), 
              metrics=['accuracy'])

# Model summary
model.summary()


c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 259, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 259, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,312 (462.16 KB)

 Trainable params: 118,312 (462.16 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Train the model
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_test, y_test))


Epoch 1/50


36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 212ms/step - accuracy: 0.4441 - loss: 1.4045 - val_accuracy: 0.4306 - val_loss: 1.5417
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 205ms/step - accuracy: 0.4703 - loss: 1.3884 - val_accuracy: 0.4271 - val_loss: 1.5479
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 207ms/step - accuracy: 0.5049 - loss: 1.3166 - val_accuracy: 0.3993 - val_loss: 1.5706
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 205ms/step - accuracy: 0.4884 - loss: 1.3209 - val_accuracy: 0.3993 - val_loss: 1.5711
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 205ms/step - accuracy: 0.4947 - loss: 1.3169 - val_accuracy: 0.4028 - val_loss: 1.5524
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 206ms/step - accuracy: 0.5035 - loss: 1.2825 - val_accuracy: 0.3993 - val_loss: 1.6432
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 203ms/step - accuracy: 0.4633 - loss: 1.3787 - val_accuracy: 0.4410 - val_loss: 1.5410
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 204ms/step - accuracy: 0.5074 - loss: 1.2771 - val_accuracy: 0.4132 - val_

In [49]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.4301 - loss: 1.5773
Test Accuracy: 0.4375


In [50]:
predictions = model.predict(X_test)
predicted_classes = tf.argmax(predictions, axis=1)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step


In [51]:
df['emotion'].value_counts()

emotion
calm        192
happy       192
sad         192
angry       192
fear        192
disgust     192
surprise    192
neutral      96
Name: count, dtype: int64